# Лабораторная работа №1

## Задание 1

*Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).*

Импортируем необходимые библиотеки.

In [3]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [4]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [5]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [6]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})
X

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
115143,0,10.8,17.4,0.0,0,48.0,0,0,24.0,20.0,63.0,41.0,1029.7,1027.3,12.6,17.1,False,False
131149,1,11.6,16.9,0.0,1,31.0,1,1,9.0,11.0,65.0,69.0,1025.0,1021.3,13.7,16.8,False,True
45243,2,13.3,19.3,0.0,1,35.0,2,2,6.0,17.0,78.0,68.0,1030.7,1026.3,16.7,18.7,False,False
102122,3,15.0,18.1,1.2,2,39.0,3,3,11.0,24.0,84.0,86.0,1020.5,1021.8,17.0,15.9,True,True
6038,4,5.7,19.1,0.2,3,11.0,2,1,2.0,2.0,100.0,62.0,1028.1,1024.5,11.2,18.8,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106293,41,20.2,38.5,0.4,15,57.0,12,11,26.0,22.0,50.0,19.0,1011.7,1009.6,23.8,36.7,False,False
58446,8,11.4,22.8,0.0,7,39.0,14,10,11.0,15.0,49.0,57.0,1008.4,1009.1,16.3,16.9,False,True
96481,16,16.7,26.2,1.4,14,39.0,9,14,11.0,22.0,55.0,37.0,1001.9,1003.6,22.6,25.1,True,False
112981,21,11.0,19.9,2.0,5,37.0,0,7,15.0,17.0,74.0,63.0,1018.0,1013.9,14.4,17.1,True,False


Отделяем таргетный столбец RainTomorrow от остального датасета.

In [7]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [8]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 2

*Провести классификацию найденного датасета, методом к-ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем класс KNeighborsClassifier для классификации датасета методом k-ближайших соседей.

In [10]:
from sklearn.neighbors import KNeighborsClassifier

Создаём объект класса KNeighborsClassifier, реализующий алгоритм k-ближайших соседей.

In [11]:
kn_classifier = KNeighborsClassifier()

Определяем параметры, которые мы хотим оптимизировать для модели KNN.
* 'n_neighbors': range(1,30) - определяет количество ближайших соседей, используемых для классификации.
* 'metric': ['minkowski', 'euclidean', 'manhattan'] - определяет список метрик, которые мы хотим попробовать для измерения расстояния между точками данных.

In [12]:
kn_params ={
    'n_neighbors': range(1,30),
    'metric':['minkowski','euclidean','manhattan']  
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.
* cv=5 - определяет количество фолдов (разделений) для кросс-валидации. В данном случае 5-кратная кросс-валидация.
* n_jobs=-1 - определяет количество ядер процессора, которые будут использоваться для параллельного выполнения Grid Search.

In [13]:
kn_grid = GridSearchCV(kn_classifier, kn_params, cv=5, n_jobs=-1)
kn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': range(1, 30)})

Обучаем модель на тренировочных данных.

In [14]:
kn_best_model = KNeighborsClassifier(**kn_grid.best_params_)
kn_best_model.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=21)

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [15]:
kn_predicted = kn_best_model.predict(X_test)

Выводим отчёт.

In [16]:
print('Параметры:', kn_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, kn_predicted, digits=5))

Параметры: {'metric': 'manhattan', 'n_neighbors': 21}
Оценка:
               precision    recall  f1-score   support

       False    0.84433   0.96533   0.90079      9029
        True    0.76034   0.38192   0.50845      2600

    accuracy                        0.83490     11629
   macro avg    0.80233   0.67363   0.70462     11629
weighted avg    0.82555   0.83490   0.81307     11629

